# Tskit and R

In [1]:
library(reticulate)
msprime <- import("msprime")

ts <- msprime$sim_ancestry(5, random_seed=42)
ts

╔═══════════════════════╗
║TreeSequence           ║
╠═══════════════╤═══════╣
║Trees          │      1║
╟───────────────┼───────╢
║Sequence Length│    1.0║
╟───────────────┼───────╢
║Sample Nodes   │     10║
╟───────────────┼───────╢
║Total Size     │2.3 KiB║
╚═══════════════╧═══════╝
╔═══════════╤════╤═════════╤════════════╗
║Table      │Rows│Size     │Has Metadata║
╠═══════════╪════╪═════════╪════════════╣
║Edges      │  18│508 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Individuals│   5│ 92 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Migrations │   0│  4 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Mutations  │   0│  8 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Nodes      │  19│460 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
║Populations│   1│216 Bytes│         Yes║
╟───────────┼────┼─────────┼────────────╢
║Provenances│   1│949 Bytes│          No║
╟───────────┼────┼─────────┼────────────╢
